# 1. OpenAI Basics
<br/><br/>

## Setup

### Install dependencies

In [ ]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pydantic~=2.9 pypdf~=5.1 --upgrade --quiet
%pip install openai~=1.54 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt

### Load environment variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# import os
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")

## Chat

### Setup OpenAI client and helper function

In [ ]:
from openai import AzureOpenAI

# Create an OpenAI client, connecting to OpenAI via an Azure deployment 
client = AzureOpenAI(api_version="2024-10-01-preview")

# These models (deployments) are currently available: gpt-4o, gpt-4o-mini, o1-mini, o1-preview  
def get_completion(prompt: str=None, model: str = "gpt-4o", messages: list = None, useJson: bool = False) -> str:
    if messages is None:
        messages = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        # stop=["Paris"], # stop the completion when the model generates the word "Paris"
        # max_tokens=100, # maximum number of tokens to generate
        temperature=1.0, # 0.0: deterministic, 1.0: more creative
        response_format={ "type": "json_object" } if useJson else { "type": "text" },
    )
    return response.choices[0].message.content

### Try some basic chatting with the model

In [ ]:
question = "What is the capital of France?"
# Here we use a kind of prompt template for the user's question (can be useful sometimes)
prompt = f"""
Be very funny when answering the questions below. 

Question from user, separated by ```:  
```{question}```
"""
response = get_completion(prompt) #, model="o1-preview")
print(response)

### Let's use a system message to set the tone

In [ ]:
messages = [
    # Set the tone of the conversation, using system instructions: 
    {"role": "system", "content": "You are Marvin, the depressed robot from Hitchhiker's Guide to the Galaxy."},
    {"role": "user", "content": prompt},
]

response = get_completion(messages=messages) #, model="o1-preview")
print(response)

### Include some assistant messages, as an example of setting up a conversation history

In [ ]:
messages = [
    {"role": "system", "content": "You are Marvin, the depressed robot from Hitchhiker's Guide to the Galaxy. You absolutely hate jokes and refuse to play along and instead change the subject."},
    {"role": "user", "content": "knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "A little old lady."}
]

response = get_completion(messages=messages) #, model="o1-preview")
print(response)

### Let's try some JSON output

In [ ]:
question = "Tell me a joke about Iceland."
prompt = f"""
Be very funny when answering the questions below. 

Question from user, separated by ```:  
```{question}```
"""
response = get_completion(prompt, useJson=True) #, model="o1-preview")
print(response)

# NOTE: If you get an error, add this text to the prompt: 
# "Return the response in JSON format."